# Análisis de sentimientos con reviews de productos de Amazon España (opcional)

Si has hecho ya el ejercicio de web scraping con `Requests` y `BeautifulSoup` habrás visto cómo extraer datos de una página web.

El dataset que utilizarás en este ejercicio (que no es obligatorio entregar) lo he generado utilizando `Scrapy` y `BeautifulSoup`, y contiene unas $700.000$ entradas con dos columnas: el número de estrellas dadas por un usuario a un determinado producto y el comentario sobre dicho producto; exactamente igual que en el ejercico de scraping.

Ahora, tu objetivo es utilizar técnicas de procesamiento de lenguaje natural para hacer un clasificador que sea capaz de distinguir (¡y predecir!) si un comentario es positivo o negativo.

Es un ejercicio MUY complicado, más que nada porque versa sobre técnicas que no hemos visto en clase. Así que si quieres resolverlo, te va a tocar estudiar y *buscar por tu cuenta*; exactamente igual que como sería en un puesto de trabajo. Dicho esto, daré un par de pistas:

+ El número de estrellas que un usuario da a un producto es el indicador de si a dicho usuario le ha gustado el producto o no. Una persona que da 5 estrellas (el máximo) a un producto probablemente esté contento con él, y el comentario será por tanto positivo; mientras que cuando una persona da 1 estrella a un producto es porque no está satisfecha... 
+ Teniendo el número de estrellas podríamos resolver el problema como si fuera de regresión; pero vamos a establecer una regla para convertirlo en problema de clasificación: *si una review tiene 4 o más estrellas, se trata de una review positiva; y será negativa si tiene menos de 4 estrellas*. Así que probablemente te toque transformar el número de estrellas en otra variable que sea *comentario positivo/negativo*.

Y... poco más. Lo más complicado será convertir el texto de cada review en algo que un clasificador pueda utilizar y entender (puesto que los modelos no entienden de palabras, sino de números). Aquí es donde te toca investigar las técnicas para hacerlo. El ejercicio se puede conseguir hacer, y obtener buenos resultados, utilizando únicamente Numpy, pandas y Scikit-Learn; pero siéntete libre de utilizar las bibliotecas que quieras.

Ahora escribiré una serie de *keywords* que probablemente te ayuden a saber qué buscar:

`bag of words, tokenizer, tf, idf, tf-idf, sklearn.feature_extraction, scipy.sparse, NLTK (opcional), stemmer, lemmatizer, stop-words removal, bigrams, trigrams`

No te desesperes si te encuentras muy perdido/a y no consigues sacar nada. Tras la fecha de entrega os daré un ejemplo de solución explicado con todo el detalle posible.

¡Ánimo y buena suerte!

## Respuesta
He estado mirando por internet como hacer este problema.
Vamos a crear una matriz donde las filas sean los comentarios y las columnas, las palabras mas repetidas.
En cada celda de la matriz, obtendremos las veces que esa palabra se repite.
Esta matriz, constituirá nuestras _features_ y el _label_ será una columna de 1 y 0 donde 1 significa comentario positivo y 0 negativo.
Para realizar esto necesitamos:

* limpiar cada comentario de caracteres que no son letras
* crear un "Word of Bag", donde ponemos las ocurrencias de cada palabra y el impacto (numero de ocurrencias)
* Elegir algun clasificador que funcione bien con lenguaje natural (RandomForests,Naive-Bayes..)
* hacer la prediccion y el posterios _score_

Me he basado en este tutorial de Kaggle:

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

Y viendo que el tamaño de las matriz del WordOfBags puede ser enorme (unos 5.5 gigas para 1000 palabras), he tenido que limitar el numero de palabras a contar a 300 para llegar a buen puerto.

## procesando el dataset..

In [37]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/home/bellinsky/anaconda3/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.8 s


In [38]:
import pandas as pd
dataset=pd.read_csv("amazon_es_reviews.csv",sep=";")

time: 4min 11s


In [39]:
#añadimos una columna "positivo" y mapeamos los comentarios a 1 o 0 

time: 1.55 s


In [40]:

dataset["positivo"]=dataset.apply(lambda x: int(1) if x["estrellas"]>3 else int(0), axis=1)

dataset[0:10]


,comentario,estrellas,positivo
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,0
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,0
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1
4,A simple vista m parecia una buena camara pero...,1.0,0
5,"NI para pasar el rato, los personajes no tiene...",1.0,0
6,el fabricante decia que es compatible con la d...,2.0,0
7,"el libro está en muy buenas condiciones, pero ...",3.0,0
8,"buen aspecto, pero le falta fortaleza. util pa...",3.0,0
9,Explica de forma simple y sencilla los pensami...,5.0,1


time: 36.4 s


In [41]:
#creamos un nuevo dataset solo con las columnas comentario y positivo
ds=dataset
del ds["estrellas"]
ds[0:10]


,comentario,positivo
0,"Para chicas es perfecto, ya que la esfera no e...",1
1,Muy floja la cuerda y el anclaje es de mala ca...,0
2,"Razonablemente bien escrito, bien ambientado, ...",0
3,Hola! No suel o escribir muchas opiniones sobr...,1
4,A simple vista m parecia una buena camara pero...,0
5,"NI para pasar el rato, los personajes no tiene...",0
6,el fabricante decia que es compatible con la d...,0
7,"el libro está en muy buenas condiciones, pero ...",0
8,"buen aspecto, pero le falta fortaleza. util pa...",0
9,Explica de forma simple y sencilla los pensami...,1


time: 842 ms


In [42]:
#vamos a ver que pinta tienen los comentarios, por si hay que pasarle algun parser, imprimimos algunos de ellos..
print(ds["comentario"][0])
print(ds["comentario"][40])

Para chicas es perfecto, ya que la esfera no es muy grande y la correa se adapta a las muñecas más finas. Un pelín gordo (para mi gusto). La carga por movimiento no dura mucho. Después de 1-2 días sin llevarlo se para.
La funda es tal y como se muestra en las imágenes. Algo a mejorar: la tira para colgar debería ser un poco más consistente y quizas si tubiera 2 cremalleras seria más cómodo. Yo la compré para una canon g15 y el tamaño es ideal aunque si llevas la correa de la cámara para colgarla al cuello sin funda, en el momento de guardarla en la funda queda un poco justa.
time: 62.1 ms


solo tenemos carateres del tipo ",(;)." habrá que eliminarlos para tener un WordOfBags (WOB) limpio.

Tendremos que decidir que hacemos con las palabras que no tienen un significado "sentimental", es decir, aquellas que no conllevan juicios de valor. Para ello, existe una libreria en Python que trae listas de "stop Words" en cualquier idioma. Para ello, debemos importar la librearia NTLK (Python Natural Language Toolkit y acceder al metodo: stop_words:

In [43]:
import nltk
from nltk.corpus import stopwords
#nltk.download()  esto solo hay que hacerlo una vez
stopwords_sp=stopwords.words("spanish")  #lista de stopwords en español

time: 781 ms


## Creando las features desde el Bag of words

Ahora que tenemos los comentarios, la pregunta es:¿cómo los convertimos en alguna representación matemática compatible con Machine Learning?
Se podría contar el número de veces que una palabra se repite, por ejemplo:

"el raton está en la cocina"
"el gato persigue al raton"

de estas dos frases, nuestro vocabulario sería:
{ el, ratón, está, en, la, cocina,gato, persigue, al }

contando las veces que aparece cada palabra en cada frase, tendriamos los siguientes ocurrencias:

{2,2,1,1,1,1,1,1,1} ("el" aparece 2 veces, "ratón" otas 2 y asi..)

De esta forma, podrímos quedarnos con las palabras que mas se repiten en nuestros comentarios, para ello vamos a crear los VectorizeCounters:







In [44]:

from sklearn.feature_extraction.text import CountVectorizer
MAX_WORDS=300
# inicializamos el CountVectorizer, con un maximo de MAX_WORDS palabras
vectorizer = CountVectorizer(analyzer = "word",   
                             ngram_range=(1, 2),
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = stopwords_sp)  
#                             max_features = MAX_WORDS) #me quedo con las MAX_WORDS palabras que mas se repiten

# y finalmente creamos un array de features, que tiene tantas filas como comentarios y en cada columna
# el numero de veces que aparece cada palabra de las 100 mas repetidas
train_data_features = vectorizer.fit_transform(ds["comentario"])
print(type(train_data_features))

MemoryError: 

time: 34min 17s


vamos a ver que palabras contiene nuestro vocabulario

In [ ]:
vocab = vectorizer.get_feature_names()
print (vocab[0:500])

In [ ]:
import numpy as np
dist = np.sum(train_data_features.toarray(), axis=0)

#vamos a imprimir cada palabra unica y el numero de ocurrencias
for tag, count in zip(vocab, dist):
    print ( count, tag )

## entrenando el modelo

In [ ]:
print(type(train_data_features))
print(train_data_features.shape)

Ahora tenemos una matriz de 702446 filas y 100 columnas. He tenido que restringir el numero de palabras a 100 puesto que con 5000 se va de memoria. 
Nos queda unicamente concatenar a esta matriz, el vector de positivo/negativo y llamar a algun clasificador:

In [ ]:
import scipy.sparse as sp
from scipy.sparse import csr_matrix

In [ ]:
#como el train_data_features es una matriz disperas, usamos los metodos de la libreria scipy.sparse para concatenar ambas matrices
vector_positivo=csr_matrix(ds["positivo"]).T

sp_dataset=sp.hstack([train_data_features,vector_positivo]).toarray()
#comprobamos que lo hemos hecho bien, observando la forma de cada matriz
print(vector_positivo.shape)
print(train_data_features.shape)
print(sp_dataset.shape)

In [ ]:
from sklearn.model_selection import train_test_split

datos_spliteados = train_test_split(sp_dataset,
                                    train_size=0.8, # 80% training
                                    test_size=0.2   # 20% testing
                                   )

In [ ]:
#separamos los datos en train y test
train=datos_spliteados[0]
test=datos_spliteados[1]

train_label=train[:,-1]
test_label=test[:,-1]

train_features= np.delete(train, MAX_WORDS, axis=1)
test_features=np.delete(test,MAX_WORDS,axis=1)

In [ ]:
# probamos ahora con un logistic regressor
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr = lr.fit(test_features, test_label)
print("bien clasificados en test: %.16f" % lr.score(X=test_features,y=test_label))



In [36]:
# probamos ahora con un Naives-Bayes Classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(test_features, test_label)
print("bien clasificados en test: %.16f" % gnb.score(X=test_features,y=test_label))



MemoryError: 

time: 26.7 s


In [ ]:
# Probamos con un RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 

forest = forest.fit( train_features, train_label )
print("Bien clasificados en test: %.16f" % forest.score(X=test_features,y=test_label))